<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Nodes-checking" data-toc-modified-id="Nodes-checking-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Nodes checking</a></span></li><li><span><a href="#Nodes_tags" data-toc-modified-id="Nodes_tags-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Nodes_tags</a></span><ul class="toc-item"><li><span><a href="#Auditing-phase" data-toc-modified-id="Auditing-phase-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Auditing phase</a></span></li><li><span><a href="#Exploring-phase" data-toc-modified-id="Exploring-phase-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Exploring phase</a></span></li></ul></li></ul></li><li><span><a href="#Overview-of-the-Data" data-toc-modified-id="Overview-of-the-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Overview of the Data</a></span></li><li><span><a href="#OSM-to-CSV-convertion" data-toc-modified-id="OSM-to-CSV-convertion-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>OSM to CSV convertion</a></span></li><li><span><a href="#SQL-section" data-toc-modified-id="SQL-section-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>SQL section</a></span></li><li><span><a href="#Other-ideas-about-the-datasets" data-toc-modified-id="Other-ideas-about-the-datasets-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Other ideas about the datasets</a></span></li><li><span><a href="#Literature" data-toc-modified-id="Literature-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Literature</a></span></li></ul></div>

4. Sczytanie
4. Podać czy jest aktualne, ponieważ Wrocław jest miastem w ciągłej rozbudowie i to bardzo istotne
5. Liczba przedszkoli
6. Liczba fastfoodów
7. 

In [24]:
import xml.etree.ElementTree as ET
import os
import re
import pyprind
import osmcsv
import sqlite3
import pandas as pd

from collections import defaultdict
from __future__ import division
from tablecreator import table_creator

# Setting file
wroclaw = 'wroclaw'

# Introduction

Problem encountered :

1. Due to the application OSM as the GPS backend I would like to see whether it is not outdated.
2. Due to the problem with importing data from csv files, I have created an own importer based on the sqlite3 python library. 


In [2]:
def count_tags(filename):
    tags = defaultdict(lambda: 0)
    for i in ET.iterparse(filename):
        tags[i[1].tag] += 1
    return tags

In [3]:
tags = count_tags(wroclaw)

In [7]:
tag_summary = pd.DataFrame.from_dict(tags, orient='index')
tag_summary.columns = ["Number of elements"]
tag_summary

Number of elements
way                    39294
member                 68743
osm                        1
note                       1
tag                   214770
bounds                     1
nd                    360671
node                  253769
meta                       1
relation                1290

In [90]:
def osm_update_street(tag = None,
               ):
    
    ''''This function changes data with the updator function. It should be included into the
    osmcsv.py file before stage of the saving the csv.  
    
    [INPUT]
    keys - list of keys to be searched with the spcific value;
    pattern - regex pattern;
    
    [OUTPUT]
    None - it changes the string before including it into the csv file. 
    '''
    if tag['k'].keys()[0] in ["street"]:
        name = tag['k']["street"]
        word = re.compile('(^")(.*)("$)')
        tag['k']["street"] = word.findall(name)[0][1]
        print(tag['k']["street"])
        

   

## Nodes checking

After importing data to csv I have found some issues - the id which should be a primary key is repetetive:

```nodes.csv:253769: INSERT failed: UNIQUE constraint failed: nodes.id```


Let's eliminate problems. 

From schema we have:

In [ ]:
table_creator = """CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);

In the pythonic way

Name | Type of data SQL | Type of data python 
:-:|-|-
id| integer | int [primary key]
lat | REAL | float
lon | REAL | float
user | TEXT | int
uid | INTEGER | int
version | INTEGER | int
changeset | INTEGER | int
timestamp | TEXT | string


In the file schema we have however:

```'version': {'required': True, 'type': 'string'},```

What is inconsistent with the integer type, let's try to convert the string to int:

In [116]:
for _, elem in ET.iterparse('wroclaw'):
    if elem.tag == "node":
        try:
            int(elem.attrib["version"])
        except:
            print("Problem")
            break

Python is able to convert it to the integer so it cannot be the issue. 

Let's then see if the ids are unique. 

In [118]:
unique_id = set()
n = 0 
for _, elem in ET.iterparse('wroclaw'):
    if elem.tag == "node":
        if elem.attrib["id"] in unique_id:
            print("Problem in the node No.{}".format(n))
            break
        else:
            unique_id.add(elem.attrib["id"])
            n += 1
    
    

In [120]:
len(unique_id)

253769

We will check it manually during uploading each node separately from node.csv into the sqlite3:

In [180]:
nodes = pd.read_csv('nodes.csv')

In [181]:
nodes.head()

id        lat        lon     user      uid  version  changeset  \
0   95180666  51.133562  17.047751  maraf24  2098272        6   34674314   
1   95180677  51.132926  17.041397  maraf24  2098272        6   34674314   
2  145744978  51.099329  17.032203  Ventlan   717069        7   32843769   
3  145744984  51.101038  17.025219      lms   117617        5   11048634   
4  145744994  51.101283  17.020169  Ventlan   717069        7   32846165   

              timestamp  
0  2015-10-16T12:23:16Z  
1  2015-10-16T12:23:16Z  
2  2015-07-24T10:15:56Z  
3  2012-03-21T06:25:02Z  
4  2015-07-24T10:40:10Z

In [183]:
for num, i in enumerate(range(nodes.shape[0])):
    id_key, lat, lon, user, uid, version, changeset, timestamp = nodes.iloc[i]

    c.execute('''INSERT INTO nodes 
                (id, lat, lon, user, uid, version, changeset, timestamp)
        VALUES ({},{},{},'{}',{},{},{},'{}')'''.format(id_key, 
                                                   lat, 
                                                   lon, 
                                                   user, 
                                                   uid, 
                                                   version, 
                                                   changeset, 
                                                   timestamp)
         )
    
conn.commit()
conn.close()

```
sqlite> SELECT user, count(user) FROM nodes GROUP BY user ORDER BY count(user) DESC LIMIT 30;```
```
rowers2|84148
lms|74261
maraf24|65038
Ancymon|6468
psboypl|4955
Zmianowy|2039
mrIimbsf|1982
marek kleciak|1837
Nolan Proost|1674
miko101|842
one_half_3544|819
jacek_zwik|811
Podolak|652
VRO|506
Jaszczur|426
Ventlan|421
emes|391
RitterR|368
tomwyr|346
Dotevo|343
tajemniczyjogurt|283
alfredlampe|236
maro21|234
Yarl|213
Jedrzej Pelka|179
LafarM|167
lukaszww|165
zoidberg|162
nyuriks|154
DJG_ww|135```


There are three guys that more than 60k nodes! It is 88% of the nodes!. 

```
sqlite> SELECT count(user) FROM nodes;```

```
253769```


In [193]:
(84148 + 74261 + 65038)/253769

0.8805133802789151

## Nodes_tags

We have not got any problems with importing data.

### Auditing phase

```
sqlite> select key, count(key) from nodes_tags GROUP BY key ORDER by count(key) DESC LIMIT 20;```

```height,23177
OBJECTID_height_lidar,11469
street,4765
housenumber,4740
city,4698
postcode,4546
street:sym_ul,4266
country,3892
name,2886
highway,1911
amenity,1877
barrier,1639
natural,1403
opening_hours,782
shop,740
addr:date,642
addr:id,642
addr:postcode:id,642
addr:layer,641
addr:postcode:layer,641```


What is interesting in the Polish section there is a unique key addded to each street ("street: sym_ul"). As you are probably  an English speaking reader you may google translate this thread: https://forum.openstreetmap.org/viewtopic.php?id=59111 or here (https://wiki.openstreetmap.org/wiki/Talk:Pl:Importy/Adresy). 

Sym_ul is the id added by the Polish National Statistical Office. 

From the above we could see that  there are 4765 street tags, however street:sym_ul have 4266 positions. 

Let's check which street does not have any sym_ul tag: 

```
sqlite> SELECT count(*) FROM (SELECT value FROM nodes_tags WHERE nodes_tags.key = "street" GROUP BY value);

292
```

```
sqlite> SELECT count(*) FROM (SELECT value FROM nodes_tags WHERE nodes_tags.key = "street:sym_ul" GROUP BY value);

290
```




Then - let's compare list of streets with the sym_ul. 

In [3]:
nodes_tags = pd.read_csv("nodes_tags.csv")
ways_tags = pd.read_csv("ways_tags.csv")

In [4]:
nodes_tag_street= pd.DataFrame(nodes_tags[nodes_tags.key=="street"])

In [5]:
nodes_tag_sym_ul = pd.DataFrame(nodes_tags[nodes_tags.key=="street:sym_ul"])

In [6]:
street_code={}

In [7]:
weirds = []

for num, i in zip(nodes_tag_street.index, nodes_tag_street.id):
    checker = 0
    for num_sec, j in zip(nodes_tag_sym_ul.index, nodes_tag_sym_ul.id):
        if i == j:
            street_code[nodes_tag_street.get_value(num, "value")] = nodes_tag_sym_ul.get_value(num_sec, "value")
            checker = 1
            break
            
    if checker:
        continue
    weirds.append(nodes_tag_street.get_value(num, "value"))

In [8]:
len(weirds) == nodes_tag_street.shape[0] - nodes_tag_sym_ul.shape[0]

True

In [9]:
for i in weirds:
    try: 
        street_code[i]
    except KeyError:
        print(i)

Plac Konstytucji 3 Maja
Plac Konstytucji 3 Maja
Grabarska


In [10]:
pd.concat([nodes_tags[nodes_tags.value == "Grabarska"],
          nodes_tags[nodes_tags.value == "Plac Konstytucji 3 Maja"]])

id     key                    value  type
49828  4273252219  street                Grabarska  addr
47071  3772479530  street  Plac Konstytucji 3 Maja  addr
49795  4264746322  street  Plac Konstytucji 3 Maja  addr

In [11]:
ways_tags_street= pd.DataFrame(ways_tags[ways_tags.key=="street"])

In [12]:
ways_tags_sym_ul = pd.DataFrame(ways_tags[ways_tags.key=="street:sym_ul"])

In [13]:
for num, i in zip(ways_tags_street.index, ways_tags_street.id):
    checker = 0
    for num_sec, j in zip(ways_tags_sym_ul.index, ways_tags_sym_ul.id):
        if i == j:
            street_code[ways_tags_street.get_value(num, "value")] = ways_tags_sym_ul.get_value(num_sec, "value")
            checker = 1
            break
            
    if checker:
        continue
    weirds.append(ways_tags_street.get_value(num, "value"))

In [15]:
for i in weirds:
    try: 
        street_code[i]
    except KeyError:
        print(i)

Grabarska
Adama Mickiewicza
Na Niskich Łąkach
Grabarska


In [16]:
pd.concat([nodes_tags[nodes_tags.value == "Grabarska"],
          nodes_tags[nodes_tags.value == "Plac Konstytucji 3 Maja"]])

id     key                    value  type
49828  4273252219  street                Grabarska  addr
47071  3772479530  street  Plac Konstytucji 3 Maja  addr
49795  4264746322  street  Plac Konstytucji 3 Maja  addr

### Exploring phase

Let's check number of kindergarden in the neighbourhood.

```
sqlite> select *  from nodes_tags WHERE key = "amenity" AND value LIKE "k%";
```

```
1168111478,amenity,kindergarten,regular
1200503957,amenity,kindergarten,regular
1375026875,amenity,kindergarten,regular
3834091857,amenity,kindergarten,regular
5016353990,amenity,kindergarten,regular
```

```
sqlite> select *  from nodes_tags WHERE key = "amenity" AND value LIKE "fast_food";
```

```
87
```

It is not good place for my kid. 

```
sqlite> select *  from nodes_tags WHERE key = "amenity" AND value LIKE "k%";
```
```
1168111478,amenity,kindergarten,regular
1200503957,amenity,kindergarten,regular
1375026875,amenity,kindergarten,regular
3834091857,amenity,kindergarten,regular
5016353990,amenity,kindergarten,regular
```

# Overview of the Data

# OSM to CSV convertion

# SQL section

In [11]:
pattern = re.compile("(.+)[a-zA-Z]{1}(.+)[a-zA-Z]{1}")

In [12]:
word ="2017-08-24T09:35:12Z"

In [13]:
pattern.findall(word)

[('2017-08-24', '09:35:12')]

# Other ideas about the datasets

# Literature

1. On Wrocław https://en.wikipedia.org/wiki/Wroc%C5%82aw